In [351]:
import nltk
import numpy as np
import string
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [352]:
def initialiseModel(): 
    tb = pd.read_excel('textBooksDataPreprocessed.xlsx')
    docs_arr = tb['preprocessed'].tolist()
    tfidf = TfidfVectorizer()
    tfidf = tfidf.fit(docs_arr)
    docs_vecs = tfidf.transform(docs_arr).toarray()

    return docs_vecs, tfidf, tb


def preprocessing(docs):
    stop_words = stopwords.words('english')
    stemmer = nltk.stem.PorterStemmer()
    punc = str.maketrans('','', string.punctuation)
    doc_no_punc = docs.translate(punc).lower()
    
    return doc_no_punc

In [353]:
def make_similarity_dataframe(similarity_matrix):
	column_labels = [str(i) for i in range(len(similarity_matrix[0]))]
	row_labels = ["query" + str(i) for i in range(len(similarity_matrix))]

	return pd.DataFrame(similarity_matrix, index=row_labels, columns=column_labels)
	

def get_top_5_recommendations_from_consine_similarity(textbooks, series, order): 
	sorted_series = series.sort_values(ascending=False)
	docsID = textbooks['book-id']
	docsName = textbooks['title']

	for i in sorted_series[:5].index: 
		pos = int(i)	
		order += str(docsID.iloc[pos]) + ","
		textbooks.drop(textbooks.index[[pos]], inplace = True)

	return(order, textbooks)

In [354]:
#adds to order based on genre given. drops those books from remaining_textbooks
def get_books_in_genre(remaining_textbooks, genre, order): 
    books_in_genre = remaining_textbooks[remaining_textbooks['genre']==genre]
    for i in list(books_in_genre['book-id']): 
        order += str(i) + ","
    remaining_textbooks.drop(books_in_genre.index, inplace = True)
    return remaining_textbooks, order 

def get_all_remaining_books(remaining_textbooks, order): 
    for i in list(remaining_textbooks['book-id']): 
        order += str(i) + ","
    return order

# def gets_books_by_similarity(remaining_textbooks, eventText, order,):  

In [357]:
(docs_vecs, tfidf, textbooks) = initialiseModel()

tbs = textbooks.copy() 
interest = "Math/Stats, Computer Sciences"
_eventLatest = "Math"
order = "" 

#order by event title/cosine similarity
_eventLatest = preprocessing(_eventLatest)
query_vec = tfidf.transform([_eventLatest]).toarray() 
query_vec.shape

# gives us a N_QUERY_DOCS X N_DOCS matrix
docs_similarity = cosine_similarity(query_vec, docs_vecs)

# docs_similarity
df = make_similarity_dataframe(docs_similarity)

(order, tbs) = get_top_5_recommendations_from_consine_similarity(tbs, df.iloc[0], order)

#order by genre interest 
if 'Computer Sciences' in interest: 
    (tbs, order) = get_books_in_genre(tbs, 'Computer Sciences', order)
if 'Math/Stats' in interest: 
    (tbs, order) = get_books_in_genre(tbs, 'Math/Stats', order)
if 'Humanities' in interest: 
    (tbs, order) = get_books_in_genre(tbs, 'Humanities', order)
if 'Biological/Physical Sciences' in interest: 
    (tbs, order) = get_books_in_genre(tbs, 'Biological/Physical Sciences', order)
if 'Business' in interest: 
    (tbs, order) = get_books_in_genre(tbs, 'Business', order)
if 'Others' in interest: 
    (tbs, order) = get_books_in_genre(tbs, 'Others', order)

#finalise order with remaining books
order = get_all_remaining_books(tbs, order)
order = order[:-1]
print(order)


215,32,343,20,301,28,183,189,192,220,244,316,375,376,18,19,27,36,68,69,72,83,86,87,88,91,92,93,137,151,156,281,284,298,307,312,343,358,359,360,361,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,21,22,23,24,25,26,29,30,31,33,34,35,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,70,71,73,74,75,76,77,78,79,80,81,82,84,85,89,90,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,138,139,140,141,142,143,144,145,146,147,148,149,150,152,153,154,155,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,184,185,186,187,188,190,191,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,216,217,218,219,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,